In [12]:
%matplotlib inline
import qutip as qt
import numpy as np
import scipy
from scipy import constants
import itertools
import math

import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from systemConst import Tunabletransmon,QQ,iniState1Qsys,iniState2Qsys
from CZwave3 import CZpulse

pi = np.pi
e = constants.e
h = constants.h
hbar = constants.hbar
ep0 = constants.epsilon_0
mu0 = constants.mu_0
opts = qt.solver.Options(nsteps=10000)

In [13]:
#savepath = 'C:/Sota_Ino/Scripts/TestKankyo/cQEDSim/core/' #PC in TUS
savepath = 'C:/Users/Sota/Scripts/TestKankyo/cQEDSim/core/'

In [14]:
####### Hamiltonian term generators #######
def ket(Nq, i):
    return qt.basis(Nq, i)

def jj(Nq, lv):
    ket = qt.basis(Nq, lv)
    return ket * ket.dag()

def Delta_q(wqs:list, wd, j):
    return wqs[j] - j * wd

def Hq(Nq, wqs:list):
    Hqs = 0
    for i in range(Nq):
        Hqs = Hqs + wqs[i] * jj(Nq, i)
    return Hqs
    
def Hq_rot(Nq, wqs:list, wd):
    Hqr = 0
    for i in range(Nq):
        D = Delta_q(wqs, wd, i)
        Hqr = Hqr + D * jj(Nq, i)
    return Hqr

def _jj1(Nq, i):
    # for taransmon's destroy operator
    ket_low = qt.basis(Nq, i)
    bra_high = qt.basis(Nq, i+1).dag()
    return ket_low * bra_high

def c(Nq):
    # taransmon's destroy operator
    cc = 0
    for i in range(Nq-1):
        cc = cc + np.sqrt(i + 1) * _jj1(Nq, i)
    return cc

## Parameters
---

In [15]:
###### qubit [GHz] ######
Nq = 4
Q1 = Tunabletransmon(f01=5.286,alpha=-0.318,Nq=3)
Q2 = Tunabletransmon(f01=5.759,alpha=-0.298,Nq=3)
wq1s=Q1.enes
wq2s=Q2.enes
fq1=wq1s[1]
fq2=wq2s[1]

J=0.015 #[GHz]
QQ=QQ(Q1,Q2,J)
print(fq1+abs(Q2.anh))

5.584


## Hamiltonian assembling
---

In [16]:
#最近接バイアス
Q1_m_I1 = -2.14 #Q1側のI1の下端
Q1_p_I1 = 2.44  #Q1側のI1の上端
Q2_m_I2 = -1.22 #Q2側のI2の下端
Q2_p_I2 = 1.43  #Q2側のI2の上端

#クロストーク
Q1_m_I2 = -3.06
Q1_p_I2 = 3.54
Q2_m_I1 = -2.989
Q2_p_I1 = 3.693

pifluxquanta_1 = Q1_p_I1-Q1_m_I1
pifluxquanta_2 = Q2_p_I2-Q2_m_I2
m11 = pi/pifluxquanta_1
m22 = pi/pifluxquanta_2
currentoffset1 = (Q1_p_I1+Q1_m_I1)/2
currentoffset2 = (Q2_p_I2+Q2_m_I2)/2

print('πΦ_1={:.2f}[mA]'.format(pifluxquanta_1))
print('πΦ_2={:.2f}[mA]'.format(pifluxquanta_2))
print('m11:{:.3f}[Φ/mA]'.format(m11))
print('m22:{:.3f}[Φ/mA]'.format(m22))
print('currentoffset1={:.3f}[mA]'.format(currentoffset1))
print('currentoffset2={:.3f}[mA]'.format(currentoffset2))

pifluxquanta_1_2 = Q1_p_I2-Q1_m_I2
pifluxquanta_2_1 = Q2_p_I1-Q2_m_I1
m12 = pi/pifluxquanta_1_2
m21 = pi/pifluxquanta_2_1
currentoffset12 = (Q1_p_I1+Q1_m_I1)/2
currentoffset21 = (Q2_p_I2+Q2_m_I2)/2

print('πΦ_12={:.2f}[mA]'.format(pifluxquanta_1_2))
print('πΦ_21={:.2f}[mA]'.format(pifluxquanta_2_1))
print('m11:{:.3f}[Φ/mA]'.format(m11))
print('m22:{:.3f}[Φ/mA]'.format(m22))
print('currentoffset1={:.3f}[mA]'.format(currentoffset1))
print('currentoffset2={:.3f}[mA]'.format(currentoffset2))

πΦ_1=4.58[mA]
πΦ_2=2.65[mA]
m11:0.686[Φ/mA]
m22:1.186[Φ/mA]
currentoffset1=0.150[mA]
currentoffset2=0.105[mA]
πΦ_12=6.60[mA]
πΦ_21=6.68[mA]
m11:0.686[Φ/mA]
m22:1.186[Φ/mA]
currentoffset1=0.150[mA]
currentoffset2=0.105[mA]


In [21]:
import japanize_matplotlib

keys={0:['00','b'],1:['10','y'],2:['01','r'],3:['20','k'],4:['11','m'],5:['02','c'],6:['12','#a65628'],7:['21','#ff7f00']}
currentoffset2 = 1.175 #[mA]
current=np.linspace(currentoffset2-1,currentoffset2+1,101)
#current = np.linspace(-0.7,0.7,141)
enemap=np.zeros((Nq**2,len(current)))

# Q2側のバイアスのみ動かす
for k,cur in enumerate(current):
    #mag1 = 0.476/pi*cur+fluxoffset1
    mag1 = 0
    mag2 = cur #1.186/pi*(cur-currentoffset2)
    Q1 = Tunabletransmon(f01=5.284,alpha=-0.318,Nq=3,phi=0)
    Q2 = Tunabletransmon(f01=5.759,alpha=-0.298,Nq=3,phi=mag2)
    Hq1 = Q1.Hlab
    Hq2 = Q2.Hlab #  - self.Iq2*0
    QQHam=Hq1 + Hq2 + Hint
    ene=np.array([QQHam.eigenenergies()])
    enes=[ene[0][i] - ene[0][0] for i in range(len(ene[0]))]
    for count in range(0,Nq**2):
        enemap[count][k]=enes[count]
            
plt.figure()
"""
for count2 in range(0,Nq**2-1):
    if keys[count2][0]=='20':
        plt.plot(current,enemap[count2],label=keys[count2][0],color='g')
    if keys[count2][0]=='11':
        plt.plot(current,enemap[count2],label=keys[count2][0],color='red')
    if keys[count2][0]=='02':
        plt.plot(current,enemap[count2],label=keys[count2][0],color='purple')
"""
for count2 in range(0,Nq**2-1):
    plt.plot(current,enemap[count2],label=keys[count2][0])

#ene_sa = np.abs(ene10-ene01)
#curr = current[np.argmin(ene_sa)]
#print(curr)
#plt.vlines(curr,0,10)
#plt.xlim([0.0,0.2])
#plt.ylim([10.2,11.1])
plt.xlabel('電流バイアス[mA]',fontsize=18)
plt.ylabel('周波数[GHz]',fontsize=20)
plt.legend(loc='upper left',fontsize=15)
#plt.yticks([10.4, 11],fontsize=20)
#plt.xticks([0.0,0.05, 0.10, 0.15, 0.20],fontsize=20)
plt.yticks(fontsize=20)
plt.show()

NameError: name 'Nb1' is not defined